In [ ]:
!pip install -U ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.0/617.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4

In [ ]:
import torch
import os
from pathlib import Path
import cv2
from timeit import default_timer as timer
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

# Download Dataset and Models

In [ ]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/U4mt8nk_jyImH-5vZZamHK0Yz6QJyTUNgsPMXJWwQao
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/U4mt8nk_jyImH-5vZZamHK0Yz6QJyTUNgsPMXJWwQao'
API Key valid
Gitlab access token valid
Saved details successfully!


In [ ]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023

phase2_train_v0.csv: 100% 553k/553k [00:00<00:00, 1.67MB/s]
phase2_train_v0.zip:  36% 3.72G/10.3G [09:15<14:12, 7.73MB/s]

In [ ]:
!unzip -qq test_images_phase1.zip -d test_images_phase1/

In [ ]:
# Downloading the weight file at https://drive.google.com/file/d/1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS/view
!gdown "1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS"

In [ ]:
TEST_DATA_DIR = 'test_images_phase1/'

In [ ]:
def classify_image(image):
    image_information = {}
    result = yolov5_model(image)
    result_df = result.pandas().xyxy[0]
    if result_df.empty:
        print('No results from yolov5 model!')
    else:
        image_information = result_df.to_dict()
    return image_information

In [ ]:
# getting mosquito_class name from predicted result
def extract_predicted_mosquito_class_name(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("name").get(0))
    return mosquito_class

In [ ]:
# getting mosquito_class number from predicted result
def extract_predicted_mosquito_class_number(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("class").get(0))
    return mosquito_class

In [ ]:
# getting mosquito_class confidence score from predicted result
def extract_predicted_mosquito_class_confidence(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("confidence").get(0))
    return mosquito_class

In [ ]:
def extract_predicted_mosquito_bbox(extractedInformation):
    bbox = []
    if extractedInformation is not None:
        xmin = str(extractedInformation.get("xmin").get(0))
        ymin = str(extractedInformation.get("ymin").get(0))
        xmax = str(extractedInformation.get("xmax").get(0))
        ymax = str(extractedInformation.get("ymax").get(0))
        bbox = [xmin, ymin, xmax, ymax]
    return bbox

# find image id
def find_image_id(original_image):
    image_name = os.path.splitext(original_image)[0]
    return image_name

In [ ]:
# path to dataset
root_images = os.path.join(TEST_DATA_DIR)

all_images = os.listdir(root_images)
print(f"Total images: {len(all_images)}")

class_labels = {
    "aegypti":      0,
    "albopictus":   1,
    "anopheles":    2,
    "culex":        3,
    "culiseta":     4,
    "japonicus/koreicus": 5
}

# counter for correctly recognized mosquito classes
counter = 0
labels_counter = 0
no_mosquito_detected = 0
rows = []

# yolov5 challenge trained baseline
trained_model_pretrained = "mosquitoalert-yolov5-baseline.pt"

# yolov5_model
# Colab gets an error on this -> Use Runtime -> Restart Session (The data will not be deleted)
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path = trained_model_pretrained, force_reload = True)

In [ ]:

for original_image in tqdm(all_images):
    try:
        original_image_file = os.path.join(root_images, original_image)
        # checking if it is a file
        if os.path.isfile(original_image_file):
            # opening testing image
            # print(f'You are watching: {original_image}')
            # classifying image by yolov5 model
            predictedInformation = classify_image(original_image_file)
            mosquito_class_name_predicted = ""
            mosquito_class_number_predicted = ""
            mosquito_class_confidence = 0
            mosquito_class_bbox = [0, 0, 0, 0]

            if predictedInformation:
                mosquito_class_name_predicted = extract_predicted_mosquito_class_name(predictedInformation)
                mosquito_class_number_predicted = extract_predicted_mosquito_class_number(predictedInformation)
                mosquito_class_confidence = extract_predicted_mosquito_class_confidence(predictedInformation)
                mosquito_class_bbox = extract_predicted_mosquito_bbox(predictedInformation)
                counter += 1
            else:
                no_mosquito_detected += 1
            # print(f"Predicted mosquito class: {mosquito_class_name_predicted} with {float(mosquito_class_confidence):.2f} confidence score.")
            #  bbox = [xmin, ymin, xmax, ymax]
            # create row for csv file
            row = [original_image, mosquito_class_name_predicted, mosquito_class_number_predicted, mosquito_class_confidence,
                   mosquito_class_bbox[0], mosquito_class_bbox[1], mosquito_class_bbox[2], mosquito_class_bbox[3]]
            rows.append(row)
            labels_counter += 1
            # print(f"Finished file reading, file {original_image} read correctly!")
    except Exception as e:
        print(f'Unable to process file: {original_image}!')
        print(f'Exception: {e}!')

In [ ]:
df = pd.DataFrame(rows, columns=["img_fName", "class_label", "predicted_class_number", "confidence_score", "bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr"])
sub_df = df.drop(['predicted_class_number', 'confidence_score'], axis=1, inplace=False)
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)